In [62]:
!pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 1.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import time
import re
import datetime as dt
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import sin,cos,arccos,pi,round
import math
import warnings
warnings.filterwarnings('ignore')

## 지역 측정
### 지역의 위치에 대해서 측정

In [2]:
loc_PM = pd.read_csv('./META/pmmap.csv', encoding = 'UTF-8')
loc_PM.head()

,Location,Latitude,Longitude,Description
0,아름동,36.512252,127.246789,세종특별자치시 보듬3로 114 아름동커뮤니티센터 옥상 (아름동)
1,신흥동,36.592887,127.292550,세종 조치원읍 군청로 87-16(신흥동) 세종특별자치시 조치원청사 옥상
2,노은동,36.368242,127.318498,대전 유성구 노은동로 87번길 89(노은1동 주민센터) 노은1동 주민센터 3층 옥상
3,문창동,36.317215,127.437825,대전 중구 보문로 20번길 38(문창동 주민센터) 문창동 주민센터
4,읍내동,36.372388,127.417714,대전 대덕구 대전로 1331번길 75(태아산업(주)) 태아산업(주)


In [3]:
loc_PM.sort_values(by=['Location'], ascending=True,inplace=True)
loc_PM.set_index('Location',inplace=True)

In [4]:
#Description Drop -> 스크린 공간을 너무 많이 잡아 먹어서 삭제 
loc_PM.drop('Description',axis = 1,inplace = True)

In [5]:
loc_PM.head()

,Latitude,Longitude
Location,,
공주,36.446951,127.119209
노은동,36.368242,127.318498
논산,36.199217,127.087021
대천2동,36.353148,126.589735
독곶리,36.987579,126.391672


In [6]:
loc_AWS = pd.read_csv('./META/awsmap.csv', index_col = 'Location', encoding = 'UTF-8')
loc_AWS.head()

,Latitude,Longitude,Description
Location,,,
오월드,36.2913,127.3959,대전광역시 중구사정공원로 70 오월드 내 플라워랜드
세천,36.3402,127.4938,대전광역시 동구세천동 63-1
장동,36.4135,127.4382,대전광역시 대덕구장동 360-2
세종고운,36.5315,127.2406,세종특별자치시 고운동산 25번지
세종금남,36.4585,127.2688,세종특별자치시 시군구금남면 성덕리 77-3


In [7]:
#Description Drop -> 스크린 공간을 너무 많이 잡아 먹어서 삭제 
loc_AWS.drop('Description',axis = 1,inplace = True)

In [8]:
loc_AWS.head()

,Latitude,Longitude
Location,,
오월드,36.2913,127.3959
세천,36.3402,127.4938
장동,36.4135,127.4382
세종고운,36.5315,127.2406
세종금남,36.4585,127.2688


In [9]:
# Train data import
PM_loc = []

for nm_PM in loc_PM.index:
    pm = pd.read_csv(f'./TRAIN/{nm_PM}.csv',encoding = 'UTF-8')
    PM_loc.append(pm)
    globals()['PM_{}'.format(nm_PM)] = pm
    
for nm_AWS in loc_AWS.index:
    globals()['AWS_{}'.format(nm_AWS)]=pd.read_csv(f'./TRAIN_AWS/{nm_AWS}.csv',encoding = 'UTF-8')

In [10]:
# Test data import
TPM_loc = []

for nm_PM in loc_PM.index:
    tpm = pd.read_csv(f'./TEST_INPUT/{nm_PM}.csv',encoding = 'UTF-8')
    TPM_loc.append(tpm)
    globals()['TPM_{}'.format(nm_PM)] = tpm
    
for nm_AWS in loc_AWS.index:
    globals()['TAWS_{}'.format(nm_AWS)]=pd.read_csv(f'./TEST_AWS/{nm_AWS}.csv',encoding = 'UTF-8')

In [11]:
AWS_공주

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,공주,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,공주,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,공주,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,공주,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,공주,0.164297,0.113889,0.020460,0.0,0.750
...,...,...,...,...,...,...,...,...
35059,3,12-31 19:00,공주,0.287520,0.765833,0.071611,0.0,0.577
35060,3,12-31 20:00,공주,0.281201,0.753056,0.063939,0.0,0.586
35061,3,12-31 21:00,공주,0.260664,0.000000,0.010230,0.0,0.674
35062,3,12-31 22:00,공주,0.262243,0.011944,0.020460,0.0,0.660


In [12]:
TAWS_공주

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,4,01-01 00:00,공주,0.244866,0.123333,0.038363,0.0,0.647
1,4,01-01 01:00,공주,0.232227,0.167778,0.033248,0.0,0.648
2,4,01-01 02:00,공주,0.206951,0.000000,0.002558,0.0,0.734
3,4,01-01 03:00,공주,0.199052,0.000000,0.002558,0.0,0.753
4,4,01-01 04:00,공주,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...,...,...,...
7723,4,11-18 19:00,공주,0.445498,0.791667,0.097187,0.0,0.368
7724,4,11-18 20:00,공주,0.426540,0.781667,0.104859,0.0,0.502
7725,4,11-18 21:00,공주,0.413902,0.735278,0.048593,0.0,0.484
7726,4,11-18 22:00,공주,0.404423,0.727222,0.058824,0.0,0.564


In [13]:
AWS_공주[AWS_공주['연도']==3].head()

,연도,일시,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
26280,3,01-01 00:00,공주,0.208531,0.109722,0.015345,0.0,0.785
26281,3,01-01 01:00,공주,0.192733,0.213056,0.017903,0.0,0.824
26282,3,01-01 02:00,공주,0.178515,0.000000,0.007673,0.0,0.831
26283,3,01-01 03:00,공주,0.173776,0.000000,0.010230,0.0,0.848
26284,3,01-01 04:00,공주,0.157978,0.327500,0.017903,0.0,0.865


## 함수 정리

In [14]:
import math

def incenter(x_lst, y_lst):  # 내심
    p1_x = x_lst[0];p1_y = y_lst[0]
    p2_x = x_lst[1];p2_y = y_lst[1]
    p3_x = x_lst[2];p3_y = y_lst[2]
    l12 = math.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)
    l13 = math.sqrt((p1_x - p3_x)**2 + (p1_y - p3_y)**2)
    l23 = math.sqrt((p2_x - p3_x)**2 + (p2_y - p3_y)**2)
    X = (l12*p3_x + l13*p2_x + l23*p1_x) / (l12 + l13 + l23)
    Y = (l12*p3_y + l13*p2_y + l23*p1_y) / (l12 + l13 + l23)
    return (X,Y)

def centroid(x_lst, y_lst):   # 무게중심
    n = len(x_lst)
    X = 0;Y = 0
    X = sum(x_lst);Y = sum(y_lst)
    X /= n;Y /= n
    return (X,Y)

## aws 와의 거리 계산 함수 -> 단순 유클리드 거리가 아니라 gps상의 거리를 이용

def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

def getDistanceBetweenPointsNew(latitude1, longitude1, latitude2, longitude2, unit = 'miles'):
    theta = longitude1 - longitude2
    dist = 60 * 1.1515 * rad2deg(
        arccos(
            (sin(deg2rad(latitude1)) * sin(deg2rad(latitude2))) + 
            (cos(deg2rad(latitude1)) * cos(deg2rad(latitude2)) * cos(deg2rad(theta)))
        )
    )
    if unit == 'miles':
        return round(dist, 2)
    if unit == 'kilometers':
        return round(dist * 1.609344, 3)
def distance(p1, p2):  #거리
    p1_x = p1[0];p1_y = p1[1]
    p2_x = p2[0];p2_y = p2[1]
    D = getDistanceBetweenPointsNew(p1_x, p1_y, p2_x,p2_y, unit = 'kilometers')
    return D


In [15]:
rads = []
Near_AWS = []
Count_AWS = []
lctn_AWS = {}
for i in range(len(loc_PM.index)): # 17개 PM 관측소
    lat_PM = loc_PM['Latitude'][i]
    lon_PM = loc_PM['Longitude'][i]
    
    lats_AWS = [];lons_AWS = []
    
    near_AWS = ''
    rad = 0
    count_AWS = 0
    
    while len(near_AWS) == 0:
        rad += 2.5     # 반경을 조금씩 넓혀가며
        
        for j in range(len(loc_AWS.index)):
            lat_AWS = loc_AWS['Latitude'][j]
            lon_AWS = loc_AWS['Longitude'][j]
            if distance((lat_AWS, lon_AWS), (lat_PM, lon_PM)) < rad:
                count_AWS += 1
                lats_AWS.append(lat_AWS);lons_AWS.append(lon_AWS);
                near_AWS += str(',') + loc_AWS.index[j]
    
    rads.append(rad)
    Near_AWS.append(near_AWS[1:])
    Count_AWS.append(count_AWS)
    lctn_AWS[f'{loc_PM.index[i]}'] = [lats_AWS,lons_AWS]
    
loc_PM['Rad'] = rads
loc_PM['Near_AWS'] = Near_AWS
loc_PM['Count_AWS'] = Count_AWS
loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS
Location,,,,,
공주,36.446951,127.119209,5.0,공주,1
노은동,36.368242,127.318498,10.0,계룡,1
논산,36.199217,127.087021,2.5,논산,1
대천2동,36.353148,126.589735,10.0,대천항,1
독곶리,36.987579,126.391672,5.0,대산,1
동문동,36.780158,126.455197,15.0,태안,1
모종동,36.782700,127.014610,17.5,"성거,아산",2
문창동,36.317215,127.437825,5.0,오월드,1
성성동,36.840313,127.141777,5.0,성거,1


In [16]:
cent_lat = [];cent_lon=[];distance_PC = []   # 일단 무게중심, 위도경도만
for nm_PM in loc_PM.index:
    cent_point = centroid(lctn_AWS[nm_PM][0], lctn_AWS[nm_PM][1])
    distance_PC.append(distance(cent_point,(loc_PM.loc[nm_PM]['Latitude'],loc_PM.loc[nm_PM]['Longitude'])))
    cent_lat.append(cent_point[0]);cent_lon.append(cent_point[1])
    
loc_PM['Cent_lat'] = cent_lat
loc_PM['Cent_lon'] = cent_lon
loc_PM['PM-Cent_d'] = distance_PC

loc_PM

,Latitude,Longitude,Rad,Near_AWS,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d
Location,,,,,,,,
공주,36.446951,127.119209,5.0,공주,1,36.4828,127.13650,4.275
노은동,36.368242,127.318498,10.0,계룡,1,36.3132,127.24070,9.274
논산,36.199217,127.087021,2.5,논산,1,36.2116,127.10820,2.347
대천2동,36.353148,126.589735,10.0,대천항,1,36.3244,126.50210,8.475
독곶리,36.987579,126.391672,5.0,대산,1,37.0106,126.38810,2.579
동문동,36.780158,126.455197,15.0,태안,1,36.7585,126.29640,14.347
모종동,36.782700,127.014610,17.5,"성거,아산",2,36.8620,127.01075,8.824
문창동,36.317215,127.437825,5.0,오월드,1,36.2913,127.39590,4.735
성성동,36.840313,127.141777,5.0,성거,1,36.8782,127.15610,4.401


## Train Dataset 만들기(AWS 이용)

In [17]:
df = pd.DataFrame()

In [18]:
'''
PM_loc = [PM_공주,PM_노은동, PM_논산,PM_대천2동,PM_독곶리,PM_동문동,PM_모종동, PM_문창동,PM_성성동,PM_신방동, PM_신흥동,
    PM_아름동,PM_예산군, PM_읍내동,PM_이원면, PM_정림동, PM_홍성읍]
'''

for i,nm_PM in enumerate(loc_PM.index):
    temp = PM_loc[i]
    temp[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc[nm_PM][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
    connectaws = PM_loc[i].iloc[:,:2]
    listaws = list(loc_PM.iloc[i,3].split(','))
    connectaws['기온(°C)'] = 0
    connectaws['풍향(deg)'] = 0
    connectaws['풍속(m/s)'] = 0
    connectaws['강수량(mm)'] = 0
    connectaws['습도(%)'] = 0
    for loc in listaws:
        a = pd.read_csv(f'./TRAIN_AWS/{loc}.csv',encoding = 'UTF-8')
        connectaws['기온(°C)'] += a['기온(°C)']
        connectaws['풍향(deg)'] += a['풍향(deg)']
        connectaws['풍속(m/s)'] += a['풍속(m/s)']
        connectaws['강수량(mm)'] += a['강수량(mm)']
        connectaws['습도(%)'] += a['습도(%)']
    connectaws['기온(°C)'] /= len(listaws)
    connectaws['풍향(deg)']/= len(listaws)
    connectaws['풍속(m/s)'] /= len(listaws)
    connectaws['강수량(mm)']/= len(listaws)
    connectaws['습도(%)'] /= len(listaws)
    v = pd.merge(temp,connectaws,how='right',on=['연도','일시'])
    df = pd.concat([df,v], axis=0)
df.head()    

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0,01-01 00:00,공주,0.056,5.0,1,36.4828,127.1365,4.275,0.173776,0.201944,0.023018,0.0,0.828
1,0,01-01 01:00,공주,0.060,5.0,1,36.4828,127.1365,4.275,0.176935,0.168611,0.030691,0.0,0.831
2,0,01-01 02:00,공주,0.068,5.0,1,36.4828,127.1365,4.275,0.180095,0.087222,0.033248,0.0,0.784
3,0,01-01 03:00,공주,0.060,5.0,1,36.4828,127.1365,4.275,0.178515,0.087222,0.025575,0.0,0.745
4,0,01-01 04:00,공주,0.068,5.0,1,36.4828,127.1365,4.275,0.164297,0.113889,0.020460,0.0,0.750


In [19]:
df.tail()

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
35059,3,12-31 19:00,홍성읍,0.060,5.0,1,36.6277,126.6451,3.456,0.273302,0.832222,0.086957,0.0,0.671
35060,3,12-31 20:00,홍성읍,0.052,5.0,1,36.6277,126.6451,3.456,0.271722,0.831667,0.043478,0.0,0.692
35061,3,12-31 21:00,홍성읍,0.044,5.0,1,36.6277,126.6451,3.456,0.268562,0.832500,0.066496,0.0,0.706
35062,3,12-31 22:00,홍성읍,0.052,5.0,1,36.6277,126.6451,3.456,0.262243,0.866944,0.043478,0.0,0.725
35063,3,12-31 23:00,홍성읍,0.060,5.0,1,36.6277,126.6451,3.456,0.257504,0.000000,0.000000,0.0,0.710


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596088 entries, 0 to 35063
Data columns (total 14 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   연도         596088 non-null  int64  
 1   일시         596088 non-null  object 
 2   측정소        596088 non-null  object 
 3   PM2.5      580546 non-null  float64
 4   Rad        596088 non-null  float64
 5   Count_AWS  596088 non-null  int64  
 6   Cent_lat   596088 non-null  float64
 7   Cent_lon   596088 non-null  float64
 8   PM-Cent_d  596088 non-null  float64
 9   기온(°C)     585026 non-null  float64
 10  풍향(deg)    585026 non-null  float64
 11  풍속(m/s)    585026 non-null  float64
 12  강수량(mm)    585026 non-null  float64
 13  습도(%)      585026 non-null  float64
dtypes: float64(10), int64(2), object(2)
memory usage: 68.2+ MB


In [21]:

## 달 ,날짜, 시간 구분해주기 -> 전부 int 정수형으로 
def time_spliter(df):


    df['달']=df['일시'].str.split("-").apply(lambda x: x[0]).apply(lambda x: int(x))
    
    df['날짜'] = df['일시'].str.split("-").apply(lambda x: x[1]).str.split(" ").apply(lambda x: x[0]).str.split(":").apply(lambda x: x[0]).apply(lambda x: int(x))
    df['시간'] = df['일시'].str.split("-").apply(lambda x: x[1]).str.split(" ").apply(lambda x: x[1]).str.split(":").apply(lambda x: x[0]).apply(lambda x: int(x))
    df.drop('일시',axis = 1,inplace = True)



## 주기성을 반영하기 위해서 달과 시간을 삼각함수에 집어 넣기 
# 참조
# https://dacon.io/competitions/official/235985/codeshare/7042?page=1&dtype=recent 

def time2sin_converter(df,col,a): ## 날짜 변환 

    df[f"sin {col}"] = df[col].apply(lambda x: math.sin(2*math.pi*x/a))

def time2cos_converter(df,col,a): ## 시간 변환 

    df[f"cos {col}"] = df[col].apply(lambda x: math.cos(2*math.pi*x/a))

def total_time_converter(df):

    time2sin_converter(df,'날짜',31)
    time2cos_converter(df,'날짜',31)

    time2sin_converter(df,'달',12)
    time2cos_converter(df,'달',12)

    time2sin_converter(df,'시간',24)
    time2cos_converter(df,'시간',24)

In [22]:
time_spliter(df)

In [23]:
total_time_converter(df)

In [24]:
df.drop(['달','날짜','시간'],axis=1, inplace=True)

In [25]:
df.columns

Index(['연도', '측정소', 'PM2.5', 'Rad', 'Count_AWS', 'Cent_lat', 'Cent_lon',
       'PM-Cent_d', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)',
       'sin 날짜', 'cos 날짜', 'sin 달', 'cos 달', 'sin 시간', 'cos 시간'],
      dtype='object')

In [26]:
df = df[['연도','sin 달', 'cos 달','sin 날짜', 'cos 날짜', 'sin 시간', 'cos 시간', '측정소', 'Rad', 'Count_AWS', 'Cent_lat', 'Cent_lon',
       'PM-Cent_d', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)','PM2.5']]
df.head()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,0.5,0.866025,0.201299,0.97953,0.000000,1.000000,공주,5.0,1,36.4828,127.1365,4.275,0.173776,0.201944,0.023018,0.0,0.828,0.056
1,0,0.5,0.866025,0.201299,0.97953,0.258819,0.965926,공주,5.0,1,36.4828,127.1365,4.275,0.176935,0.168611,0.030691,0.0,0.831,0.060
2,0,0.5,0.866025,0.201299,0.97953,0.500000,0.866025,공주,5.0,1,36.4828,127.1365,4.275,0.180095,0.087222,0.033248,0.0,0.784,0.068
3,0,0.5,0.866025,0.201299,0.97953,0.707107,0.707107,공주,5.0,1,36.4828,127.1365,4.275,0.178515,0.087222,0.025575,0.0,0.745,0.060
4,0,0.5,0.866025,0.201299,0.97953,0.866025,0.500000,공주,5.0,1,36.4828,127.1365,4.275,0.164297,0.113889,0.020460,0.0,0.750,0.068


In [27]:
loc = list(set(df['측정소']))
loc

['이원면',
 '홍성읍',
 '정림동',
 '노은동',
 '대천2동',
 '아름동',
 '논산',
 '동문동',
 '문창동',
 '공주',
 '독곶리',
 '신방동',
 '신흥동',
 '읍내동',
 '성성동',
 '예산군',
 '모종동']

In [28]:
from sklearn.preprocessing import LabelEncoder
observations = loc
encoder = LabelEncoder()
encoder.fit(observations)
df['측정소'] = encoder.transform(df['측정소'])

In [29]:
df.head()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,0,0.5,0.866025,0.201299,0.97953,0.000000,1.000000,0,5.0,1,36.4828,127.1365,4.275,0.173776,0.201944,0.023018,0.0,0.828,0.056
1,0,0.5,0.866025,0.201299,0.97953,0.258819,0.965926,0,5.0,1,36.4828,127.1365,4.275,0.176935,0.168611,0.030691,0.0,0.831,0.060
2,0,0.5,0.866025,0.201299,0.97953,0.500000,0.866025,0,5.0,1,36.4828,127.1365,4.275,0.180095,0.087222,0.033248,0.0,0.784,0.068
3,0,0.5,0.866025,0.201299,0.97953,0.707107,0.707107,0,5.0,1,36.4828,127.1365,4.275,0.178515,0.087222,0.025575,0.0,0.745,0.060
4,0,0.5,0.866025,0.201299,0.97953,0.866025,0.500000,0,5.0,1,36.4828,127.1365,4.275,0.164297,0.113889,0.020460,0.0,0.750,0.068


## TEST 데이터 영역

In [30]:
tdf = pd.DataFrame()

In [31]:
'''
TPM_loc = [TPM_공주,TPM_노은동, TPM_논산,TPM_대천2동,TPM_독곶리,TPM_동문동,TPM_모종동, TPM_문창동,TPM_성성동,TPM_신방동, TPM_신흥동,
    TPM_아름동,TPM_예산군, TPM_읍내동,TPM_이원면, TPM_정림동, TPM_홍성읍]
'''
TPM_loc[0]

,연도,일시,측정소,PM2.5
0,4,01-01 00:00,공주,0.060
1,4,01-01 01:00,공주,0.064
2,4,01-01 02:00,공주,0.072
3,4,01-01 03:00,공주,0.064
4,4,01-01 04:00,공주,0.056
...,...,...,...,...
7723,4,11-18 19:00,공주,0.052
7724,4,11-18 20:00,공주,0.056
7725,4,11-18 21:00,공주,0.036
7726,4,11-18 22:00,공주,0.044


In [32]:
for i,nm_PM in enumerate(loc_PM.index):
    temp = TPM_loc[i]
    temp[['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']] = loc_PM.loc[nm_PM][['Rad','Count_AWS','Cent_lat','Cent_lon','PM-Cent_d']]
    connectaws = TPM_loc[i].iloc[:,:2]
    listaws = list(loc_PM.iloc[i,3].split(','))
    connectaws['기온(°C)'] = 0
    connectaws['풍향(deg)'] = 0
    connectaws['풍속(m/s)'] = 0
    connectaws['강수량(mm)'] = 0
    connectaws['습도(%)'] = 0
    err = 0
    for loc in listaws:
        a = pd.read_csv(f'./TEST_AWS/{loc}.csv',encoding = 'UTF-8')
        a.fillna(method='ffill',inplace=True)
        connectaws['기온(°C)'] += a['기온(°C)']
        connectaws['풍향(deg)'] += a['풍향(deg)']
        connectaws['풍속(m/s)'] += a['풍속(m/s)']
        connectaws['강수량(mm)'] += a['강수량(mm)']
        connectaws['습도(%)'] += a['습도(%)']
    connectaws['기온(°C)'] /= (len(listaws))
    connectaws['풍향(deg)']/= len(listaws)
    connectaws['풍속(m/s)'] /= len(listaws)
    connectaws['강수량(mm)']/= len(listaws)
    connectaws['습도(%)'] /= len(listaws)
    v = pd.merge(temp,connectaws,how='right',on=['연도','일시'])
    tdf = pd.concat([tdf,v], axis=0)
tdf.head()    

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,4,01-01 00:00,공주,0.060,5.0,1,36.4828,127.1365,4.275,0.244866,0.123333,0.038363,0.0,0.647
1,4,01-01 01:00,공주,0.064,5.0,1,36.4828,127.1365,4.275,0.232227,0.167778,0.033248,0.0,0.648
2,4,01-01 02:00,공주,0.072,5.0,1,36.4828,127.1365,4.275,0.206951,0.000000,0.002558,0.0,0.734
3,4,01-01 03:00,공주,0.064,5.0,1,36.4828,127.1365,4.275,0.199052,0.000000,0.002558,0.0,0.753
4,4,01-01 04:00,공주,0.056,5.0,1,36.4828,127.1365,4.275,0.189573,0.000000,0.002558,0.0,0.795


In [33]:
sample = tdf[tdf['PM2.5'].isnull()]
sample.head()

,연도,일시,측정소,PM2.5,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
48,4,01-03 00:00,공주,NaN,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
49,4,01-03 01:00,공주,NaN,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
50,4,01-03 02:00,공주,NaN,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
51,4,01-03 03:00,공주,NaN,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
52,4,01-03 04:00,공주,NaN,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707


In [34]:
time_spliter(tdf)

In [35]:
total_time_converter(tdf)

In [36]:
tdf.drop(['달','날짜','시간'],axis=1, inplace=True)

In [37]:
tdf = tdf[['연도','sin 달', 'cos 달','sin 날짜', 'cos 날짜', 'sin 시간', 'cos 시간', '측정소', 'Rad', 'Count_AWS', 'Cent_lat', 'Cent_lon',
       'PM-Cent_d', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)', '습도(%)','PM2.5']]
tdf.head()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,4,0.5,0.866025,0.201299,0.97953,0.000000,1.000000,공주,5.0,1,36.4828,127.1365,4.275,0.244866,0.123333,0.038363,0.0,0.647,0.060
1,4,0.5,0.866025,0.201299,0.97953,0.258819,0.965926,공주,5.0,1,36.4828,127.1365,4.275,0.232227,0.167778,0.033248,0.0,0.648,0.064
2,4,0.5,0.866025,0.201299,0.97953,0.500000,0.866025,공주,5.0,1,36.4828,127.1365,4.275,0.206951,0.000000,0.002558,0.0,0.734,0.072
3,4,0.5,0.866025,0.201299,0.97953,0.707107,0.707107,공주,5.0,1,36.4828,127.1365,4.275,0.199052,0.000000,0.002558,0.0,0.753,0.064
4,4,0.5,0.866025,0.201299,0.97953,0.866025,0.500000,공주,5.0,1,36.4828,127.1365,4.275,0.189573,0.000000,0.002558,0.0,0.795,0.056


In [38]:
tdf['측정소'] = encoder.transform(tdf['측정소'])

In [39]:
tdf.head()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
0,4,0.5,0.866025,0.201299,0.97953,0.000000,1.000000,0,5.0,1,36.4828,127.1365,4.275,0.244866,0.123333,0.038363,0.0,0.647,0.060
1,4,0.5,0.866025,0.201299,0.97953,0.258819,0.965926,0,5.0,1,36.4828,127.1365,4.275,0.232227,0.167778,0.033248,0.0,0.648,0.064
2,4,0.5,0.866025,0.201299,0.97953,0.500000,0.866025,0,5.0,1,36.4828,127.1365,4.275,0.206951,0.000000,0.002558,0.0,0.734,0.072
3,4,0.5,0.866025,0.201299,0.97953,0.707107,0.707107,0,5.0,1,36.4828,127.1365,4.275,0.199052,0.000000,0.002558,0.0,0.753,0.064
4,4,0.5,0.866025,0.201299,0.97953,0.866025,0.500000,0,5.0,1,36.4828,127.1365,4.275,0.189573,0.000000,0.002558,0.0,0.795,0.056


In [40]:
df.dropna(axis=0, inplace=True)

In [41]:
x_train = df.iloc[:, :-1]
y_train = df.iloc[:, -1]

In [42]:
write =tdf[tdf['PM2.5'].isnull()]
write.head()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),PM2.5
48,4,0.5,0.866025,0.571268,0.820763,0.000000,1.000000,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707,NaN
49,4,0.5,0.866025,0.571268,0.820763,0.258819,0.965926,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707,NaN
50,4,0.5,0.866025,0.571268,0.820763,0.500000,0.866025,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707,NaN
51,4,0.5,0.866025,0.571268,0.820763,0.707107,0.707107,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707,NaN
52,4,0.5,0.866025,0.571268,0.820763,0.866025,0.500000,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707,NaN


In [43]:
tdf.dropna(axis=0, inplace=True)

In [44]:
x_test = tdf.iloc[:, :-1]
y_test = tdf.iloc[:, -1]

In [45]:
x_test.describe()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
count,53040.0,5.304000e+04,5.304000e+04,53040.000000,53040.000000,5.304000e+04,5.304000e+04,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000,53040.000000
mean,4.0,1.256904e-02,-1.294625e-01,0.028558,-0.027470,-7.400545e-17,-7.400859e-17,8.000000,7.794118,1.117647,36.579318,126.972015,5.962765,0.605663,0.433338,0.045261,0.002281,0.700200
std,0.0,7.389223e-01,6.611307e-01,0.708367,0.704745,7.071134e-01,7.071134e-01,4.899026,4.362511,0.322193,0.238451,0.387012,3.528858,0.161159,0.319307,0.039413,0.015744,0.207055
min,4.0,-1.000000e+00,-1.000000e+00,-0.998717,-0.994869,-1.000000e+00,-1.000000e+00,0.000000,2.500000,1.000000,36.211600,126.296400,2.210000,0.142180,0.000000,0.000000,0.000000,0.079000
25%,4.0,-8.660254e-01,-8.660254e-01,-0.724793,-0.758758,-7.071068e-01,-7.071068e-01,4.000000,5.000000,1.000000,36.324400,126.645100,3.101000,0.489731,0.124722,0.017903,0.000000,0.554000
50%,4.0,1.224647e-16,-1.836970e-16,0.101168,-0.050649,6.123234e-17,-6.123234e-17,8.000000,5.000000,1.000000,36.566700,127.136500,4.735000,0.620853,0.442708,0.035806,0.000000,0.728000
75%,4.0,8.660254e-01,5.000000e-01,0.724793,0.688967,7.071068e-01,7.071068e-01,12.000000,10.000000,1.000000,36.758500,127.240700,8.475000,0.740916,0.715833,0.063939,0.000000,0.880000
max,4.0,1.000000e+00,8.660254e-01,0.998717,1.000000,1.000000e+00,1.000000e+00,16.000000,17.500000,2.000000,37.010600,127.438200,14.347000,0.938389,1.000000,0.434783,0.647727,0.988000


## 간단한 선형 모델 이용

In [46]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression, Ridge, Lasso
lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
ridge_reg = Ridge()
ridge_reg.fit(x_train, y_train)
lasso_reg = Lasso()
lasso_reg.fit(x_train, y_train)
models = [lr_reg, ridge_reg, lasso_reg]
for i,model in enumerate(models):
    pred = model.predict(x_test)
    print('{}의 mae:'.format(i), mae(pred,y_test))

0의 mae: 0.03730291084636818
1의 mae: 0.03729788568009677
2의 mae: 0.04676660306838667


In [47]:
result = write.iloc[:,:-1]

In [48]:
result.fillna(method='ffill')

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
48,4,0.5,0.866025,0.571268,0.820763,0.000000,1.000000,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
49,4,0.5,0.866025,0.571268,0.820763,0.258819,0.965926,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
50,4,0.5,0.866025,0.571268,0.820763,0.500000,0.866025,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
51,4,0.5,0.866025,0.571268,0.820763,0.707107,0.707107,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
52,4,0.5,0.866025,0.571268,0.820763,0.866025,0.500000,0,5.0,1,36.4828,127.1365,4.275,0.230648,0.145833,0.017903,0.0,0.707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7675,4,-0.5,0.866025,-0.101168,-0.994869,-0.965926,0.258819,16,5.0,1,36.6277,126.6451,3.456,0.461295,0.783056,0.025575,0.0,0.982
7676,4,-0.5,0.866025,-0.101168,-0.994869,-0.866025,0.500000,16,5.0,1,36.6277,126.6451,3.456,0.461295,0.783056,0.025575,0.0,0.982
7677,4,-0.5,0.866025,-0.101168,-0.994869,-0.707107,0.707107,16,5.0,1,36.6277,126.6451,3.456,0.461295,0.783056,0.025575,0.0,0.982
7678,4,-0.5,0.866025,-0.101168,-0.994869,-0.500000,0.866025,16,5.0,1,36.6277,126.6451,3.456,0.461295,0.783056,0.025575,0.0,0.982


In [49]:
ans = pd.read_csv('answer_sample.csv')

In [50]:
print(len(ans.index))

78336


In [51]:
answer = ridge_reg.predict(result)

In [52]:
result.describe()

,연도,sin 달,cos 달,sin 날짜,cos 날짜,sin 시간,cos 시간,측정소,Rad,Count_AWS,Cent_lat,Cent_lon,PM-Cent_d,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
count,78336.0,7.833600e+04,7.833600e+04,78336.000000,78336.000000,7.833600e+04,7.833600e+04,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000,78336.000000
mean,4.0,1.251002e-02,-1.324386e-01,0.030503,-0.029431,-7.400849e-17,-7.401062e-17,8.000000,7.794118,1.117647,36.579318,126.972015,5.962765,0.565688,0.312125,0.032559,0.001659,0.801493
std,0.0,7.490318e-01,6.490512e-01,0.712154,0.700752,7.071113e-01,7.071113e-01,4.899011,4.362498,0.322192,0.238450,0.387011,3.528847,0.157213,0.303804,0.036833,0.012246,0.146911
min,4.0,-1.000000e+00,-1.000000e+00,-0.998717,-0.994869,-1.000000e+00,-1.000000e+00,0.000000,2.500000,1.000000,36.211600,126.296400,2.210000,0.178515,0.000000,0.000000,0.000000,0.293000
25%,4.0,-8.660254e-01,-8.660254e-01,-0.651372,-0.758758,-7.071068e-01,-7.071068e-01,4.000000,5.000000,1.000000,36.324400,126.645100,3.101000,0.451817,0.000000,0.007673,0.000000,0.702750
50%,4.0,1.224647e-16,-1.836970e-16,0.101168,-0.050649,6.123234e-17,-6.123234e-17,8.000000,5.000000,1.000000,36.566700,127.136500,4.735000,0.578989,0.249167,0.020460,0.000000,0.847000
75%,4.0,8.660254e-01,5.000000e-01,0.724793,0.688967,7.071068e-01,7.071068e-01,12.000000,10.000000,1.000000,36.758500,127.240700,8.475000,0.698262,0.537847,0.043478,0.000000,0.921625
max,4.0,1.000000e+00,8.660254e-01,0.998717,1.000000,1.000000e+00,1.000000e+00,16.000000,17.500000,2.000000,37.010600,127.438200,14.347000,0.829384,0.989167,0.414322,0.193182,0.988000


In [53]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78336 entries, 48 to 7679
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연도         78336 non-null  int64  
 1   sin 달      78336 non-null  float64
 2   cos 달      78336 non-null  float64
 3   sin 날짜     78336 non-null  float64
 4   cos 날짜     78336 non-null  float64
 5   sin 시간     78336 non-null  float64
 6   cos 시간     78336 non-null  float64
 7   측정소        78336 non-null  int32  
 8   Rad        78336 non-null  float64
 9   Count_AWS  78336 non-null  int64  
 10  Cent_lat   78336 non-null  float64
 11  Cent_lon   78336 non-null  float64
 12  PM-Cent_d  78336 non-null  float64
 13  기온(°C)     78336 non-null  float64
 14  풍향(deg)    78336 non-null  float64
 15  풍속(m/s)    78336 non-null  float64
 16  강수량(mm)    78336 non-null  float64
 17  습도(%)      78336 non-null  float64
dtypes: float64(15), int32(1), int64(2)
memory usage: 11.1 MB


In [54]:
ans.head()

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,NaN
1,4,01-03 01:00,공주,NaN
2,4,01-03 02:00,공주,NaN
3,4,01-03 03:00,공주,NaN
4,4,01-03 04:00,공주,NaN


In [55]:
ans['PM2.5']= answer

In [56]:
ans.head()

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.096669
1,4,01-03 01:00,공주,0.097469
2,4,01-03 02:00,공주,0.098211
3,4,01-03 03:00,공주,0.098843
4,4,01-03 04:00,공주,0.099323


In [57]:
ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78336 entries, 0 to 78335
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      78336 non-null  int64  
 1   일시      78336 non-null  object 
 2   측정소     78336 non-null  object 
 3   PM2.5   78336 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.4+ MB


In [58]:
ans.to_csv(path_or_buf='sss.csv')

## XGB 이용

In [59]:
def evaluate_regr(y,pred):
    mae_val = mae(y,pred)
    print(f'MAE:{mae_val}')

In [60]:
def get_model_predict(model,x_train,x_test,y_train,y_test,is_expm1=False):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    if is_expm1:
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)
    evaluate_regr(y_test,pred)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor 
xgb_reg = XGBRegressor(n_estimators=500)
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)
get_model_predict(xgb_reg,x_train,x_test,y_train_log,y_test_log,is_expm1=True)

In [ ]:
log_answer= xgb_reg.predict(result)

In [ ]:
answer = np.expm1(log_answer)

In [ ]:
ans['PM2.5']= answer

In [ ]:
ans.head()

In [ ]:
ans.to_csv(path_or_buf='XGB.csv')